![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Spark NLP Basics

## 1. Start Spark Session

In [4]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)


Spark NLP version 2.5.2
Apache Spark version: 2.4.5

In [5]:
#!pip install spark-nlp==2.4.5

import sparknlp

sparknlp.version()

Out[28]: '2.5.2'

In [6]:
!pip install spark-nlp==2.4.5

`sparknlp.start()` will start or get SparkSession with predefined parameters hardcoded in `spark-nlp/python/sparknlp/__init__.py`. here is what is going on on the background when you run `sparknlp.start()`

In [8]:
# repo >> spark-nlp/python/sparknlp/__init__.py

from pyspark.sql import SparkSession

def start(gpu=False):
    builder = SparkSession.builder \
        .appName("Spark NLP") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "1000M")
    if gpu:
        builder.config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp-gpu_2.11:2.4.5")
    else:
        builder.config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5")
        
    return builder.getOrCreate()


If you want to start `SparkSession` with your own parameters or you need to load the required jars/packages from your local disk, or you have no internet connection (that would be needed to pull the required packages from internet), you can skip `sparknlp.start()` and start your session manually as shown below.

In [10]:
"""
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark NLP Enterprise 2.4.5") \
    .master("local[8]") \
    .config("spark.driver.memory","12G") \
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "800M")\
    .config("spark.jars", "{}spark-nlp-2.4.5.jar,{}spark-nlp-jsl-2.4.5.jar".format(jar_path,jar_path)) \
    .getOrCreate()
    
"""

Out[31]: '\nfrom pyspark.sql import SparkSession\n\nspark = SparkSession.builder .appName("Spark NLP Enterprise 2.4.5") .master("local[8]") .config("spark.driver.memory","12G") .config("spark.driver.maxResultSize", "2G") .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") .config("spark.kryoserializer.buffer.max", "800M") .config("spark.jars", "{}spark-nlp-2.4.5.jar,{}spark-nlp-jsl-2.4.5.jar".format(jar_path,jar_path)) .getOrCreate()\n \n'

## 2. Using Pretrained Pipelines

https://github.com/JohnSnowLabs/spark-nlp-models

In [13]:
from sparknlp.pretrained import PretrainedPipeline

In [14]:
testDoc = '''
Peter is a very good persn.
My life in Russia is very intersting.
John and Peter are brothrs. However they don't support each other that much.
Lucas Nogal Dunbercker is no longer happy. He has a good car though.
Europe is very culture rich. There are huge churches! and big houses!
'''

### Explain Document ML

**Stages**
- DocumentAssembler
- SentenceDetector
- Tokenizer
- Lemmatizer
- Stemmer
- Part of Speech
- SpellChecker (Norvig)

In [17]:
pipeline = PretrainedPipeline('explain_document_ml', lang='en')

explain_document_ml download started this may take some time.
Approx size to download 9.4 MB
[ | ][OK!]

In [18]:
##%%time
result = pipeline.annotate(testDoc)

In [19]:
result.keys()

Out[36]: dict_keys(['document', 'spell', 'pos', 'lemmas', 'token', 'stems', 'sentence'])

In [20]:
result['sentence']

Out[37]: ['Peter is a very good persn.',
 'My life in Russia is very intersting.',
 'John and Peter are brothrs.',
 "However they don't support each other that much.",
 'Lucas Nogal Dunbercker is no longer happy.',
 'He has a good car though.',
 'Europe is very culture rich.',
 'There are huge churches!',
 'and big houses!']

In [21]:
result['token']

Out[38]: ['Peter',
 'is',
 'a',
 'very',
 'good',
 'persn',
 '.',
 'My',
 'life',
 'in',
 'Russia',
 'is',
 'very',
 'intersting',
 '.',
 'John',
 'and',
 'Peter',
 'are',
 'brothrs',
 '.',
 'However',
 'they',
 "don't",
 'support',
 'each',
 'other',
 'that',
 'much',
 '.',
 'Lucas',
 'Nogal',
 'Dunbercker',
 'is',
 'no',
 'longer',
 'happy',
 '.',
 'He',
 'has',
 'a',
 'good',
 'car',
 'though',
 '.',
 'Europe',
 'is',
 'very',
 'culture',
 'rich',
 '.',
 'There',
 'are',
 'huge',
 'churches',
 '!',
 'and',
 'big',
 'houses',
 '!']

In [22]:
list(zip(result['token'], result['pos']))

Out[39]: [('Peter', 'NNP'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('very', 'RB'),
 ('good', 'JJ'),
 ('persn', 'NN'),
 ('.', '.'),
 ('My', 'PRP$'),
 ('life', 'NN'),
 ('in', 'IN'),
 ('Russia', 'NNP'),
 ('is', 'VBZ'),
 ('very', 'RB'),
 ('intersting', 'JJ'),
 ('.', '.'),
 ('John', 'NNP'),
 ('and', 'CC'),
 ('Peter', 'NNP'),
 ('are', 'VBP'),
 ('brothrs', 'NNS'),
 ('.', '.'),
 ('However', 'RB'),
 ('they', 'PRP'),
 ("don't", 'VBP'),
 ('support', 'VB'),
 ('each', 'DT'),
 ('other', 'JJ'),
 ('that', 'IN'),
 ('much', 'JJ'),
 ('.', '.'),
 ('Lucas', 'NNP'),
 ('Nogal', 'NNP'),
 ('Dunbercker', 'NNP'),
 ('is', 'VBZ'),
 ('no', 'DT'),
 ('longer', 'RB'),
 ('happy', 'JJ'),
 ('.', '.'),
 ('He', 'PRP'),
 ('has', 'VBZ'),
 ('a', 'DT'),
 ('good', 'JJ'),
 ('car', 'NN'),
 ('though', 'IN'),
 ('.', '.'),
 ('Europe', 'NNP'),
 ('is', 'VBZ'),
 ('very', 'RB'),
 ('culture', 'RB'),
 ('rich', 'JJ'),
 ('.', '.'),
 ('There', 'EX'),
 ('are', 'VBP'),
 ('huge', 'JJ'),
 ('churches', 'NNS'),
 ('!', '.'),
 ('and', 'CC'),
 ('big', 'JJ'),
 ('houses', 'NNS'),
 ('!', '.')]

In [23]:
list(zip(result['token'], result['lemmas'], result['stems'], result['spell']))

Out[40]: [('Peter', 'Peter', 'peter', 'Peter'),
 ('is', 'be', 'i', 'is'),
 ('a', 'a', 'a', 'a'),
 ('very', 'very', 'veri', 'very'),
 ('good', 'good', 'good', 'good'),
 ('persn', 'person', 'person', 'person'),
 ('.', '.', '.', '.'),
 ('My', 'My', 'my', 'My'),
 ('life', 'life', 'life', 'life'),
 ('in', 'in', 'in', 'in'),
 ('Russia', 'Russia', 'russia', 'Russia'),
 ('is', 'be', 'i', 'is'),
 ('very', 'very', 'veri', 'very'),
 ('intersting', 'interest', 'interest', 'interesting'),
 ('.', '.', '.', '.'),
 ('John', 'John', 'john', 'John'),
 ('and', 'and', 'and', 'and'),
 ('Peter', 'Peter', 'peter', 'Peter'),
 ('are', 'be', 'ar', 'are'),
 ('brothrs', 'broth', 'broth', 'broths'),
 ('.', '.', '.', '.'),
 ('However', 'However', 'howev', 'However'),
 ('they', 'they', 'thei', 'they'),
 ("don't", "don't", "don't", "don't"),
 ('support', 'support', 'support', 'support'),
 ('each', 'each', 'each', 'each'),
 ('other', 'other', 'other', 'other'),
 ('that', 'that', 'that', 'that'),
 ('much', 'much', 'much', 'much'),
 ('.', '.', '.', '.'),
 ('Lucas', 'Lucas', 'luca', 'Lucas'),
 ('Nogal', 'Nogal', 'nogal', 'Nogal'),
 ('Dunbercker', 'Dunbercker', 'dunberck', 'Dunbercker'),
 ('is', 'be', 'i', 'is'),
 ('no', 'no', 'no', 'no'),
 ('longer', 'long', 'longer', 'longer'),
 ('happy', 'happy', 'happi', 'happy'),
 ('.', '.', '.', '.'),
 ('He', 'He', 'he', 'He'),
 ('has', 'have', 'ha', 'has'),
 ('a', 'a', 'a', 'a'),
 ('good', 'good', 'good', 'good'),
 ('car', 'car', 'car', 'car'),
 ('though', 'though', 'though', 'though'),
 ('.', '.', '.', '.'),
 ('Europe', 'Europe', 'europ', 'Europe'),
 ('is', 'be', 'i', 'is'),
 ('very', 'very', 'veri', 'very'),
 ('culture', 'culture', 'cultur', 'culture'),
 ('rich', 'rich', 'rich', 'rich'),
 ('.', '.', '.', '.'),
 ('There', 'There', 'there', 'There'),
 ('are', 'be', 'ar', 'are'),
 ('huge', 'huge', 'huge', 'huge'),
 ('churches', 'church', 'church', 'churches'),
 ('!', '!', '!', '!'),
 ('and', 'and', 'and', 'and'),
 ('big', 'big', 'big', 'big'),
 ('houses', 'house', 'hous', 'houses'),
 ('!', '!', '!', '!')]

In [24]:
import pandas as pd

df = pd.DataFrame({'token':result['token'], 
                      'corrected':result['spell'], 'POS':result['pos'],
                      'lemmas':result['lemmas'], 'stems':result['stems']})
df

,token,corrected,POS,lemmas,stems
0,Peter,Peter,NNP,Peter,peter
1,is,is,VBZ,be,i
2,a,a,DT,a,a
3,very,very,RB,very,veri
4,good,good,JJ,good,good
5,persn,person,NN,person,person
6,.,.,.,.,.
7,My,My,PRP$,My,my
8,life,life,NN,life,life
9,in,in,IN,in,in


### Explain Document DL

### Recognize Entities DL

In [27]:
recognize_entities = PretrainedPipeline('recognize_entities_dl', lang='en')

recognize_entities_dl download started this may take some time.
Approx size to download 159 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [28]:
testDoc = '''
Peter is a very good persn.
My life in Russia is very intersting.
John and Peter are brothrs. However they don't support each other that much.
Lucas Nogal Dunbercker is no longer happy. He has a good car though.
Europe is very culture rich. There are huge churches! and big houses!
'''

result = recognize_entities.annotate(testDoc)

list(zip(result['token'], result['ner']))

Out[43]: [('Peter', 'B-PER'),
 ('is', 'O'),
 ('a', 'O'),
 ('very', 'O'),
 ('good', 'O'),
 ('persn', 'O'),
 ('.', 'O'),
 ('My', 'O'),
 ('life', 'O'),
 ('in', 'O'),
 ('Russia', 'B-LOC'),
 ('is', 'O'),
 ('very', 'O'),
 ('intersting', 'O'),
 ('.', 'O'),
 ('John', 'B-PER'),
 ('and', 'O'),
 ('Peter', 'B-PER'),
 ('are', 'O'),
 ('brothrs', 'O'),
 ('.', 'O'),
 ('However', 'O'),
 ('they', 'O'),
 ("don't", 'O'),
 ('support', 'O'),
 ('each', 'O'),
 ('other', 'O'),
 ('that', 'O'),
 ('much', 'O'),
 ('.', 'O'),
 ('Lucas', 'B-ORG'),
 ('Nogal', 'I-ORG'),
 ('Dunbercker', 'I-ORG'),
 ('is', 'O'),
 ('no', 'O'),
 ('longer', 'O'),
 ('happy', 'O'),
 ('.', 'O'),
 ('He', 'O'),
 ('has', 'O'),
 ('a', 'O'),
 ('good', 'O'),
 ('car', 'O'),
 ('though', 'O'),
 ('.', 'O'),
 ('Europe', 'B-LOC'),
 ('is', 'O'),
 ('very', 'O'),
 ('culture', 'O'),
 ('rich', 'O'),
 ('.', 'O'),
 ('There', 'O'),
 ('are', 'O'),
 ('huge', 'O'),
 ('churches', 'O'),
 ('!', 'O'),
 ('and', 'O'),
 ('big', 'O'),
 ('houses', 'O'),
 ('!', 'O')]

### Clean Stop Words

In [30]:
clean_stop = PretrainedPipeline('clean_stop', lang='en')

clean_stop download started this may take some time.
Approx size to download 12.4 KB
[ | ][ / ][OK!]

In [31]:
result = clean_stop.annotate(testDoc)

' '.join(result['cleanTokens'])

Out[45]: "Peter good persn . life Russia intersting . John Peter brothrs . don't support . Lucas Nogal Dunbercker longer happy . good car . Europe culture rich . huge churches ! big houses !"

### Clean Slang

In [33]:
clean_slang = PretrainedPipeline('clean_slang', lang='en')

result = clean_slang.annotate(' Whatsup bro, call me ASAP')

' '.join(result['normal'])

clean_slang download started this may take some time.
Approx size to download 21.8 KB
[ | ][ / ][OK!]
Out[46]: 'how are you friend call me as soon as possible'

**Stages**
- DocumentAssembler
- SentenceDetector
- Tokenizer
- NER (NER with GloVe, CoNLL2003 dataset)
- Lemmatizer
- Stemmer
- Part of Speech
- SpellChecker (Norvig)

In [35]:
pipeline_dl = PretrainedPipeline('explain_document_dl', lang='en')


explain_document_dl download started this may take some time.
Approx size to download 168.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [36]:
result = pipeline_dl.annotate(testDoc)

result.keys()

Out[48]: dict_keys(['entities', 'stem', 'checked', 'lemma', 'document', 'pos', 'token', 'ner', 'embeddings', 'sentence'])

In [37]:
result['entities']

Out[49]: ['Peter', 'Russia', 'John', 'Peter', 'Lucas Nogal Dunbercker', 'Europe']

In [38]:
df = pd.DataFrame({'token':result['token'], 'ner_label':result['ner'],
                      'spell_corrected':result['checked'], 'POS':result['pos'],
                      'lemmas':result['lemma'], 'stems':result['stem']})

df

,token,ner_label,spell_corrected,POS,lemmas,stems
0,Peter,B-PER,Peter,NNP,Peter,peter
1,is,O,is,VBZ,be,i
2,a,O,a,DT,a,a
3,very,O,very,RB,very,veri
4,good,O,good,JJ,good,good
5,persn,O,person,NN,person,person
6,.,O,.,.,.,.
7,My,O,My,PRP$,My,my
8,life,O,life,NN,life,life
9,in,O,in,IN,in,in


### Spell Checker

In [40]:
spell_checker = PretrainedPipeline('check_spelling', lang='en')


check_spelling download started this may take some time.
Approx size to download 892.6 KB
[ | ][ / ][OK!]

In [41]:
result = spell_checker.annotate(testDoc)

result.keys()

Out[52]: dict_keys(['document', 'sentence', 'token', 'checked'])

In [42]:
list(zip(result['token'], result['checked']))

Out[53]: [('Peter', 'Peter'),
 ('is', 'is'),
 ('a', 'a'),
 ('very', 'very'),
 ('good', 'good'),
 ('persn', 'person'),
 ('.', '.'),
 ('My', 'My'),
 ('life', 'life'),
 ('in', 'in'),
 ('Russia', 'Russia'),
 ('is', 'is'),
 ('very', 'very'),
 ('intersting', 'interesting'),
 ('.', '.'),
 ('John', 'John'),
 ('and', 'and'),
 ('Peter', 'Peter'),
 ('are', 'are'),
 ('brothrs', 'brothers'),
 ('.', '.'),
 ('However', 'However'),
 ('they', 'they'),
 ("don't", "don't"),
 ('support', 'support'),
 ('each', 'each'),
 ('other', 'other'),
 ('that', 'that'),
 ('much', 'much'),
 ('.', '.'),
 ('Lucas', 'Lucas'),
 ('Nogal', 'Nigel'),
 ('Dunbercker', 'Dunbercker'),
 ('is', 'is'),
 ('no', 'no'),
 ('longer', 'longer'),
 ('happy', 'happy'),
 ('.', '.'),
 ('He', 'He'),
 ('has', 'has'),
 ('a', 'a'),
 ('good', 'good'),
 ('car', 'car'),
 ('though', 'though'),
 ('.', '.'),
 ('Europe', 'Europe'),
 ('is', 'is'),
 ('very', 'very'),
 ('culture', 'culture'),
 ('rich', 'rich'),
 ('.', '.'),
 ('There', 'There'),
 ('are', 'are'),
 ('huge', 'huge'),
 ('churches', 'churches'),
 ('!', '!'),
 ('and', 'and'),
 ('big', 'big'),
 ('houses', 'houses'),
 ('!', '!')]

### Spell Checker DL
https://medium.com/spark-nlp/applying-context-aware-spell-checking-in-spark-nlp-3c29c46963bc

In [44]:
spell_checker_dl = PretrainedPipeline('check_spelling_dl', lang='en')

check_spelling_dl download started this may take some time.
Approx size to download 112.1 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [45]:
text = 'We will go to swimming if the ueather is nice.'

result = spell_checker_dl.annotate(text)

list(zip(result['token'], result['checked']))

Out[55]: [('We', 'We'),
 ('will', 'will'),
 ('go', 'go'),
 ('to', 'to'),
 ('swimming', 'swimming'),
 ('if', 'if'),
 ('the', 'the'),
 ('ueather', 'weather'),
 ('is', 'is'),
 ('nice', 'nice'),
 ('.', '.')]

In [46]:
result.keys()

Out[56]: dict_keys(['document', 'sentence', 'token', 'checked'])

In [47]:
# check for the different occurrences of the word "ueather"
examples = ['We will go to swimming if the ueather is nice.',\
    "I have a black ueather jacket, so nice.",\
    "I introduce you to my sister, she is called ueather."]

results = spell_checker_dl.annotate(examples)

for result in results:
  print (list(zip(result['token'], result['checked'])))

[('We', 'We'), ('will', 'will'), ('go', 'go'), ('to', 'to'), ('swimming', 'swimming'), ('if', 'if'), ('the', 'the'), ('ueather', 'weather'), ('is', 'is'), ('nice', 'nice'), ('.', '.')]
[('I', 'I'), ('have', 'have'), ('a', 'a'), ('black', 'black'), ('ueather', 'leather'), ('jacket', 'jacket'), (',', ','), ('so', 'so'), ('nice', 'nice'), ('.', '.')]
[('I', 'I'), ('introduce', 'introduce'), ('you', 'you'), ('to', 'to'), ('my', 'my'), ('sister', 'sister'), (',', ','), ('she', 'she'), ('is', 'is'), ('called', 'called'), ('ueather', 'Heather'), ('.', '.')]

In [48]:
for result in results:
  print (result['document'],'>>',[pairs for pairs in list(zip(result['token'], result['checked'])) if pairs[0]!=pairs[1]])

['We will go to swimming if the ueather is nice.'] >> [('ueather', 'weather')]
['I have a black ueather jacket, so nice.'] >> [('ueather', 'leather')]
['I introduce you to my sister, she is called ueather.'] >> [('ueather', 'Heather')]

In [49]:
# if we had tried the same with spell_checker (previous version)

results = spell_checker.annotate(examples)

for result in results:
  print (list(zip(result['token'], result['checked'])))

[('We', 'We'), ('will', 'will'), ('go', 'go'), ('to', 'to'), ('swimming', 'swimming'), ('if', 'if'), ('the', 'the'), ('ueather', 'weather'), ('is', 'is'), ('nice', 'nice'), ('.', '.')]
[('I', 'I'), ('have', 'have'), ('a', 'a'), ('black', 'black'), ('ueather', 'weather'), ('jacket', 'jacket'), (',', ','), ('so', 'so'), ('nice', 'nice'), ('.', '.')]
[('I', 'I'), ('introduce', 'introduce'), ('you', 'you'), ('to', 'to'), ('my', 'my'), ('sister', 'sister'), (',', ','), ('she', 'she'), ('is', 'is'), ('called', 'called'), ('ueather', 'weather'), ('.', '.')]

### Parsing a list of texts

In [51]:
testDoc_list = ['French author who helped pioner the science-fiction genre.',
'Verne wrate about space, air, and underwater travel before navigable aircrast',
'Practical submarines were invented, and before any means of space travel had been devised.']

testDoc_list

Out[60]: ['French author who helped pioner the science-fiction genre.',
 'Verne wrate about space, air, and underwater travel before navigable aircrast',
 'Practical submarines were invented, and before any means of space travel had been devised.']

In [52]:
result_list = pipeline.annotate(testDoc_list)

len (result_list)

Out[61]: 3

In [53]:
result_list[0]

Out[62]: {'document': ['French author who helped pioner the science-fiction genre.'],
 'spell': ['French',
 'author',
 'who',
 'helped',
 'pioneer',
 'the',
 'sciencefiction',
 'genre',
 '.'],
 'pos': ['JJ', 'NN', 'WP', 'VBD', 'NN', 'DT', 'NN', 'NN', '.'],
 'lemmas': ['French',
 'author',
 'who',
 'help',
 'pioneer',
 'the',
 'sciencefiction',
 'genre',
 '.'],
 'token': ['French',
 'author',
 'who',
 'helped',
 'pioner',
 'the',
 'science-fiction',
 'genre',
 '.'],
 'stems': ['french',
 'author',
 'who',
 'help',
 'pioneer',
 'the',
 'sciencefict',
 'genr',
 '.'],
 'sentence': ['French author who helped pioner the science-fiction genre.']}

### Using fullAnnotate to get more details

In [55]:
text = 'Peter Parker is a nice guy and lives in New York'

In [56]:
# pipeline_dl >> explain_document_dl

detailed_result = pipeline_dl.fullAnnotate(text)

In [57]:
detailed_result

Out[65]: [{'entities': [Annotation(chunk, 0, 11, Peter Parker, {'entity': 'PER', 'sentence': '0', 'chunk': '0'}),
 Annotation(chunk, 40, 47, New York, {'entity': 'LOC', 'sentence': '0', 'chunk': '1'})],
 'stem': [Annotation(token, 0, 4, peter, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 6, 11, parker, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 13, 14, i, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 16, 16, a, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 18, 21, nice, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 23, 25, gui, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 27, 29, and, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 31, 35, live, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 37, 38, in, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 40, 42, new, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 44, 47, york, {'confidence': '1.0', 'sentence': '0'})],
 'checked': [Annotation(token, 0, 4, Peter, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 6, 11, Parker, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 13, 14, is, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 16, 16, a, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 18, 21, nice, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 23, 25, guy, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 27, 29, and, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 31, 35, lives, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 37, 38, in, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 40, 42, New, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 44, 47, York, {'confidence': '1.0', 'sentence': '0'})],
 'lemma': [Annotation(token, 0, 4, Peter, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 6, 11, Parker, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 13, 14, be, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 16, 16, a, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 18, 21, nice, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 23, 25, guy, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 27, 29, and, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 31, 35, life, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 37, 38, in, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 40, 42, New, {'confidence': '1.0', 'sentence': '0'}),
 Annotation(token, 44, 47, York, {'confidence': '1.0', 'sentence': '0'})],
 'document': [Annotation(document, 0, 47, Peter Parker is a nice guy and lives in New York, {})],
 'pos': [Annotation(pos, 0, 4, NNP, {'word': 'Peter'}),
 Annotation(pos, 6, 11, NNP, {'word': 'Parker'}),
 Annotation(pos, 13, 14, VBZ, {'word': 'is'}),
 Annotation(pos, 16, 16, DT, {'word': 'a'}),
 Annotation(pos, 18, 21, JJ, {'word': 'nice'}),
 Annotation(pos, 23, 25, NN, {'word': 'guy'}),
 Annotation(pos, 27, 29, CC, {'word': 'and'}),
 Annotation(pos, 31, 35, NNS, {'word': 'lives'}),
 Annotation(pos, 37, 38, IN, {'word': 'in'}),
 Annotation(pos, 40, 42, NNP, {'word': 'New'}),
 Annotation(pos, 44, 47, NNP, {'word': 'York'})],
 'token': [Annotation(token, 0, 4, Peter, {'sentence': '0'}),
 Annotation(token, 6, 11, Parker, {'sentence': '0'}),
 Annotation(token, 13, 14, is, {'sentence': '0'}),
 Annotation(token, 16, 16, a, {'sentence': '0'}),
 Annotation(token, 18, 21, nice, {'sentence': '0'}),
 Annotation(token, 23, 25, guy, {'sentence': '0'}),
 Annotation(token, 27, 29, and, {'sentence': '0'}),
 Annotation(token, 31, 35, lives, {'sentence': '0'}),
 Annotation(token, 37, 38, in, {'sentence': '0'}),
 Annotation(token, 40, 42, New, {'sentence': '0'}),
 Annotation(token, 44, 47, York, {'sentence': '0'})],
 'ner': [Annotation(named_entity, 0, 4, B-PER, {'word': 'Peter'}),
 Annotation(named_entity, 6, 11, I-PER, {'word': 'Parker'}),
 Annot

In [59]:
detailed_result[0]['entities']

Out[66]: [Annotation(chunk, 0, 11, Peter Parker, {'entity': 'PER', 'sentence': '0', 'chunk': '0'}),
 Annotation(chunk, 40, 47, New York, {'entity': 'LOC', 'sentence': '0', 'chunk': '1'})]

In [60]:
chunks=[]
entities=[]
for n in detailed_result[0]['entities']:
        
  chunks.append(n.result)
  entities.append(n.metadata['entity']) 
    
df = pd.DataFrame({'chunks':chunks, 'entities':entities})
df    

,chunks,entities
0,Peter Parker,PER
1,New York,LOC


In [61]:
tuples = []

for x,y,z in zip(detailed_result[0]["token"], detailed_result[0]["pos"], detailed_result[0]["ner"]):

  tuples.append((int(x.metadata['sentence']), x.result, x.begin, x.end, y.result, z.result))

df = pd.DataFrame(tuples, columns=['sent_id','token','start','end','pos', 'ner'])

df


,sent_id,token,start,end,pos,ner
0,0,Peter,0,4,NNP,B-PER
1,0,Parker,6,11,NNP,I-PER
2,0,is,13,14,VBZ,O
3,0,a,16,16,DT,O
4,0,nice,18,21,JJ,O
5,0,guy,23,25,NN,O
6,0,and,27,29,CC,O
7,0,lives,31,35,NNS,O
8,0,in,37,38,IN,O
9,0,New,40,42,NNP,B-LOC


### Use pretrained match_chunk Pipeline for Individual Noun Phrase

**Stages**
- DocumentAssembler
- SentenceDetector
- Tokenizer
- Part of Speech
- Chunker

Pipeline:

- The pipeline uses regex `<DT>?<JJ>*<NN>+`
- which states that whenever the chunk finds an optional determiner (DT) followed by any number of adjectives (JJ) and then a noun (NN) then the Noun Phrase(NP) chunk should be formed.

In [64]:
pipeline = PretrainedPipeline('match_chunks', lang='en')


match_chunks download started this may take some time.
Approx size to download 4.3 MB
[ | ][ / ][ — ][ \ ][OK!]

In [65]:
result = pipeline.annotate("The book has many chapters") # single noun phrase


In [66]:
result

Out[71]: {'chunk': ['The book'],
 'document': ['The book has many chapters'],
 'pos': ['DT', 'NN', 'VBZ', 'JJ', 'NNS'],
 'token': ['The', 'book', 'has', 'many', 'chapters'],
 'sentence': ['The book has many chapters']}

In [67]:
result['chunk']

Out[72]: ['The book']

In [68]:
result = pipeline.annotate("the little yellow dog barked at the cat") #multiple noune phrases

In [69]:
result

Out[74]: {'chunk': ['the little yellow dog', 'the cat'],
 'document': ['the little yellow dog barked at the cat'],
 'pos': ['DT', 'JJ', 'JJ', 'NN', 'JJ', 'IN', 'DT', 'NN'],
 'token': ['the', 'little', 'yellow', 'dog', 'barked', 'at', 'the', 'cat'],
 'sentence': ['the little yellow dog barked at the cat']}

In [70]:
result['chunk']

Out[75]: ['the little yellow dog', 'the cat']

### Extract exact dates from referential date phrases

In [73]:
pipeline = PretrainedPipeline('match_datetime', lang='en')


match_datetime download started this may take some time.
Approx size to download 12.9 KB
[ | ][ / ][OK!]

In [74]:
result = pipeline.annotate("I saw him yesterday and he told me that he will visit us next week")

result

Out[77]: {'document': ['I saw him yesterday and he told me that he will visit us next week'],
 'sentence': ['I saw him yesterday and he told me that he will visit us next week'],
 'token': ['I',
 'saw',
 'him',
 'yesterday',
 'and',
 'he',
 'told',
 'me',
 'that',
 'he',
 'will',
 'visit',
 'us',
 'next',
 'week'],
 'date': ['2020/08/13', '2020/08/05']}

In [75]:
pipeline.fullAnnotate("I saw him yesterday and he told me that he will visit us next week")

Out[78]: [{'document': [Annotation(document, 0, 65, I saw him yesterday and he told me that he will visit us next week, {})],
 'sentence': [Annotation(document, 0, 65, I saw him yesterday and he told me that he will visit us next week, {'sentence': '0'})],
 'token': [Annotation(token, 0, 0, I, {'sentence': '0'}),
 Annotation(token, 2, 4, saw, {'sentence': '0'}),
 Annotation(token, 6, 8, him, {'sentence': '0'}),
 Annotation(token, 10, 18, yesterday, {'sentence': '0'}),
 Annotation(token, 20, 22, and, {'sentence': '0'}),
 Annotation(token, 24, 25, he, {'sentence': '0'}),
 Annotation(token, 27, 30, told, {'sentence': '0'}),
 Annotation(token, 32, 33, me, {'sentence': '0'}),
 Annotation(token, 35, 38, that, {'sentence': '0'}),
 Annotation(token, 40, 41, he, {'sentence': '0'}),
 Annotation(token, 43, 46, will, {'sentence': '0'}),
 Annotation(token, 48, 52, visit, {'sentence': '0'}),
 Annotation(token, 54, 55, us, {'sentence': '0'}),
 Annotation(token, 57, 60, next, {'sentence': '0'}),
 Annotation(token, 62, 65, week, {'sentence': '0'})],
 'date': [Annotation(date, 57, 65, 2020/08/13, {'sentence': '0'}),
 Annotation(date, 10, 18, 2020/08/05, {'sentence': '0'})]}]

In [76]:
tuples = []

for x in detailed_result[0]["token"]:

  tuples.append((int(x.metadata['sentence']), x.result, x.begin, x.end))

df = pd.DataFrame(tuples, columns=['sent_id','token','start','end'])

df

,sent_id,token,start,end
0,0,Peter,0,4
1,0,Parker,6,11
2,0,is,13,14
3,0,a,16,16
4,0,nice,18,21
5,0,guy,23,25
6,0,and,27,29
7,0,lives,31,35
8,0,in,37,38
9,0,New,40,42


### Sentiment Analysis
#### Vivek algo

In [78]:
pipeline = PretrainedPipeline('analyze_sentiment', lang='en')

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[ | ][ / ][ — ][ \ ][OK!]

In [79]:
result = pipeline.annotate("The movie I watched today was not a good one")

result['sentiment']

Out[81]: ['negative']

#### DL version (trained on imdb)

In [81]:
sentiment_imdb = PretrainedPipeline('analyze_sentimentdl_use_imdb', lang='en')

analyze_sentimentdl_use_imdb download started this may take some time.
Approx size to download 935.8 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [82]:
sentiment_imdb_glove = PretrainedPipeline('analyze_sentimentdl_glove_imdb', lang='en')

analyze_sentimentdl_glove_imdb download started this may take some time.
Approx size to download 154.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [83]:
comment = '''
It's a very scary film but what impressed me was how true the film sticks to the original's tricks; it isn't filled with loud in-your-face jump scares, in fact, a lot of what makes this film scary is the slick cinematography and intricate shadow play. The use of lighting and creation of atmosphere is what makes this film so tense, which is why it's perfectly suited for those who like Horror movies but without the obnoxious gore.
'''
result = sentiment_imdb_glove.annotate(comment)

result['sentiment']

Out[84]: ['positive']

In [84]:
sentiment_imdb_glove.fullAnnotate(comment)[0]['sentiment']

Out[85]: [Annotation(category, 0, 433, positive, {'sentence': '0', 'positive': '0.92505074', 'negative': '0.074949294'})]

#### DL version (trained on twitter dataset)

In [86]:
sentiment_twitter = PretrainedPipeline('analyze_sentimentdl_use_twitter', lang='en')

analyze_sentimentdl_use_twitter download started this may take some time.
Approx size to download 928.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [87]:
result = sentiment_twitter.annotate("The movie I watched today was not a good one")

result['sentiment']

Out[87]: ['negative']